In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autokeras as ak
import os 
import tensorflow as tf

In [2]:
data = np.load('/work/n1000000_0804_all_flat.npz')
cut_index = np.load('/work/NIO_largerthan1000_index.npy')

In [3]:
input_state = 2
target = data['delta']
if input_state == 0:
    data_all = data['ve_dune']
elif input_state == 1:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
elif input_state == 2:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune'], data['ve_t2hk'], data['vu_t2hk'], data['vebar_t2hk'], data['vubar_t2hk']])

x_train = data_all[:10000, cut_index]
y_train = target[:10000]
x_train2 = data_all[10000:900000, cut_index]
y_train2 = target[10000:900000]
x_test = data_all[900000:, cut_index]
y_test = target[900000:]

In [4]:
generate = 10
num_of_bins = len(x_train[0])
x_train_gen = np.reshape(np.random.normal(x_train, np.sqrt(x_train), size = (generate, len(x_train), num_of_bins)), (generate*len(x_train), num_of_bins))
y_train_gen = np.repeat(y_train, generate)
x_train2_gen = np.reshape(np.random.normal(x_train2, np.sqrt(x_train2), size = (generate, len(x_train2), num_of_bins)), (generate*len(x_train2), num_of_bins))
y_train2_gen = np.repeat(y_train2, generate)
x_test_gen = np.reshape(np.random.normal(x_test, np.sqrt(x_test), size = (generate, len(x_test), num_of_bins)), (generate*len(x_test), num_of_bins))
y_test_gen = np.repeat(y_test, generate)

In [ ]:
clf = ak.StructuredDataRegressor(overwrite=True, max_trials=50)
clf.fit(x_train_gen, y_train_gen,
           validation_split = 0.1,
           batch_size=64,
           epochs=50,
           verbose=1,
           shuffle = True
       )

Trial 34 Complete [00h 02m 38s]
val_loss: 10475.669921875

Best val_loss So Far: 10472.5400390625
Total elapsed time: 01h 27m 29s

Search: Running Trial #35

Hyperparameter    |Value             |Best Value So Far 
structured_data...|True              |True              
structured_data...|False             |False             
structured_data...|1                 |1                 
structured_data...|128               |16                
structured_data...|0                 |0                 
structured_data...|1024              |1024              
regression_head...|0.5               |0.5               
optimizer         |sgd               |sgd               
learning_rate     |0.001             |0.001             
structured_data...|16                |16                


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers 

In [ ]:
model = clf.export_model()
model.summary()

In [ ]:
model.fit(x_train2_gen, y_train2_gen,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

In [ ]:
model.evaluate(x_test_gen, y_test_gen)
pre_test = model.predict(x_test_gen)

In [ ]:
plt.hist(pre_test, histtype = 'step', bins = 100, label = 'prediction')
plt.hist(y_test, histtype = 'step', bins = 100, label = 'target')
plt.xlabel('delta')
plt.ylabel('number')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(pre_test, y_test, '.', alpha = 0.05)
plt.xlabel('prediction')
plt.ylabel('target')
plt.title('delta')

In [ ]:
if input_state == 0:
    modelname = '/work/Regression/models_ve_dune/0803_delta4.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 1:
    modelname = '/work/Regression/models_all_dune/0803_delta2.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 2:
    modelname = '/work/Regression/models_all/0804_delta2.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)

In [ ]:
if input_state == 0:
    modelname = '/work/Regression/models_ve_dune/0803_delta1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 1:
    modelname = '/work/Regression/models_all_dune/0803_delta1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 2:
    modelname = '/work/ML4NO/ML/Regression/perturb_cut/0804_cut1000_delta1.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)